Make anndatas per conditions (this is used as backup -- not currently part of processing) 

In [ ]:
import pandas as pd
import os
import numpy as np
import random
import itertools
from collections import Counter
import scanpy as sc
import pickle

Read in anndata with normalized expression ie. lambdas

In [23]:
a = sc.read_h5ad('/home/sanjavickovic/data/st_data/anndata_colons_norm_all.h5ad')

In [24]:
'Make unique conditions currently present in the dataset'
regions = list(np.unique([str(i) for i in a.obs.Region]))
ages = list(np.unique([str(i) for i in a.obs.Age]))
conditions = []
for i in regions:
    for h in ages:
        conditions.append(i+"_"+h)

In [25]:
'Subset anndatas as into modules script'
st_data_per_condition = dict()
st_regions_per_condition = dict()
coordinates = dict()
for c in conditions:
    a_copy = a[a.obs.conditions == c]
    a_copy.write(os.path.join('/home/sanjavickovic/data/st_data/', c+"_anndata_norm_all.h5ad"))
    
    # gets values
    st_data_per_condition[c] = a_copy.X.T
    
    #gets regions
    st_regions_per_condition[c] = a_copy.obs['annotation'].tolist()
    
    # gets coordiantes per image
    ind_coordinates = dict()
    for i in np.unique(a_copy.obs['sample']):
        a_sample = a_copy[a_copy.obs['sample'] == i]
        ind_coordinates[str(i)] = np.array([[float(i.split("_")[2]),float(i.split("_")[3])] for i in a_sample.obs.index])
    coordinates[c] = ind_coordinates
with open('/home/sanjavickovic/data/st_data/st_data_per_condition_all.pickle', 'wb') as handle:
    pickle.dump(st_data_per_condition, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('/home/sanjavickovic/data/st_data/st_regions_per_condition_all.pickle', 'wb') as handle:
    pickle.dump(st_regions_per_condition, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('/home/sanjavickovic/data/st_data/st_coordinates_per_condition_all.pkl', 'wb') as handle:
    pickle.dump(coordinates, handle, protocol=pickle.HIGHEST_PROTOCOL)